In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Vroomie.zip을 /content/drive/MyDrive에 업로드 후 아래 코드 셀 실행

(drive 디렉터리는 위에 코드 셀 실행해서 마운트하면 나타남)



---



Vroomie 디렉터리로 이동하는 코드

In [2]:
FOLDERNAME = 'Vroomie'
assert FOLDERNAME is not None, "[!] Enter the foldername."

import sys
sys.path.append('/content/drive/MyDrive/{}'.format(FOLDERNAME))

%cd /content/drive/MyDrive/$FOLDERNAME

/content/drive/MyDrive/Vroomie




---



**객체탐지 코드**

In [ ]:
!pip install ultralytics


In [ ]:
!mkdir -p yolov8_pretrained
!wget -O yolov8_pretrained/coco128.txt https://raw.githubusercontent.com/ultralytics/yolov5/master/data/coco128.yaml

In [ ]:
!pip install deep-sort-realtime

In [ ]:
# ✅ ngrok 라이브러리 설치
!pip install pyngrok --quiet

# ngrok 토큰 등록
!ngrok authtoken <ngrok_token>

**실행 코드**

In [ ]:
# ✅ Flask 서버 + ngrok 연결 + 모델 준비
import os
import torch
import numpy as np
import cv2
from PIL import Image
from flask import Flask, request
from pyngrok import ngrok
from ultralytics import YOLO
from torchvision import transforms
from deep_sort_realtime.deepsort_tracker import DeepSort

from model.lanenet.LaneNet import LaneNet

# ✅ YOLO + Tracker
yolo_model = YOLO('./yolov8_pretrained/yolov8l.pt')
tracker = DeepSort(max_age=50, nms_max_overlap=0.5)

# ✅ LaneNet 설정
DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
resize_h, resize_w = 256, 512
lanenet_model = LaneNet(arch='DeepLabv3+')
lanenet_model.load_state_dict(torch.load('./log/best_model.pth', map_location=DEVICE))
lanenet_model.eval().to(DEVICE)

# ✅ 전처리
data_transform = transforms.Compose([
    transforms.Resize((resize_h, resize_w)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
])

# ✅ 침범 감지 함수
def check_intrusion(track, binary, frame_size):
    x1, y1, x2, y2 = map(int, track.to_ltrb())
    w_frame, h_frame = frame_size
    fc = ((x1 + x2) // 2, y2)
    mx = int(fc[0] * (binary.shape[1] / w_frame))
    my = int(fc[1] * (binary.shape[0] / h_frame))
    mx, my = np.clip(mx, 0, binary.shape[1]-1), np.clip(my, 0, binary.shape[0]-1)
    pixel_val = binary[my, mx]
    on_lane = pixel_val > 128
    return on_lane

# ✅ Flask 서버 생성
app = Flask(__name__)

# ✅ ngrok 포트 열기
public_url = ngrok.connect(5000)
print(f"🔗 외부 접속 URL: {public_url}")

# ✅ 디버깅용 기본 경로
@app.route("/")
def home():
    return "✅ Flask + YOLO + LaneNet 서버 정상 작동 중"

# ✅ 실제 추론 경로
@app.route("/predict", methods=['POST'])
def predict():
    file = request.files['file']
    img_bytes = file.read()
    np_arr = np.frombuffer(img_bytes, np.uint8)
    frame = cv2.imdecode(np_arr, cv2.IMREAD_COLOR)

    # Resize
    frame = cv2.resize(frame, (640, 360))
    frame = cv2.rotate(frame, cv2.ROTATE_90_CLOCKWISE)
    h_frame, w_frame = frame.shape[:2]

    # YOLO 감지
    det = yolo_model.predict(source=[frame], save=False)[0]
    results = []
    for box in det.boxes.data.tolist():
        conf = float(box[4])
        if conf < 0.65:
            continue
        x1, y1, x2, y2 = map(int, box[:4])
        cls = int(box[5])
        results.append([[x1, y1, x2 - x1, y2 - y1], conf, cls])
    tracks = tracker.update_tracks(results, frame=frame)

    # 차선 인식
    rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    pil = Image.fromarray(rgb)
    inp = data_transform(pil).unsqueeze(0).to(DEVICE)
    out = lanenet_model(inp)
    binary = (out['binary_seg_pred'].squeeze().cpu().numpy() * 255).astype(np.uint8)

    # 침범 판단
    intrusion_detected = False
    for track in tracks:
        if not track.is_confirmed():
            continue
        if check_intrusion(track, binary, (w_frame, h_frame)):
            intrusion_detected = True
            print(f"🚨 침범 감지 - 차량 ID: {track.track_id}")
            break

    # ✅ 프레임 시각화 추가
    #for track in tracks:
     #   if not track.is_confirmed():
     #       continue
     #   x1, y1, x2, y2 = map(int, track.to_ltrb())
     #   color = (0, 0, 255) if intrusion_detected else (0, 255, 0)
     #   cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)
     #   cv2.putText(frame, f'ID:{track.track_id}', (x1, y1 - 10),
     #               cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

    # binary 미니맵 붙이기
    #bin_color = cv2.cvtColor(binary, cv2.COLOR_GRAY2BGR)
    #small = cv2.resize(bin_color, (160, 90))
    #frame[0:90, -160:] = small

    # ✅ 실제 imshow 실행
    #cv2_imshow(frame)

    return {"result": "침범" if intrusion_detected else "정상"}

# ✅ Flask 실행
app.run(port=5000)
